In [0]:
import json
import os
import pandas as pd
import dask.dataframe as da
import time
import datetime

**PREPROCESSING WEATHER DATA**

In [0]:
df_list=[]

def create_df(f):

  dat = json.load(open(f, 'rb'))
  k = dat['data']['weather']

  del dat['data']['request']   

  req_weather = ['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph', 'tempF', 'WindChillF', 'humidity', 'date', 'time', 'airport']
  hourly_keys = list(dat['data']['weather'][0]['hourly'][0].keys())
  hourly_req = [i for i in hourly_keys if i in req_weather]

  w_dict={}
  w_dict['airport'] = f[23:26]
  for i in k:
    w_dict['date'] = i['date']
    for j in i['hourly']:
        w_dict.update({key : j[key] for key in hourly_req})
        dat = pd.DataFrame(w_dict, index = [0])
        df_list.append(dat)


In [0]:
start_time = time.time()
path = 'drive/My Drive/weather'
valid_yr = ['2016','2017']
airports = os.listdir(path)

req_json=([os.path.join(path,i,j) for i in airports for j in os.listdir(os.path.join(path,i)) if(j[:4] in valid_yr)])

for file in req_json:
  create_df(file)

weather_dataframe=pd.concat(df_list) 

In [0]:
weather_dataframe.to_csv("drive/My Drive/weather_final.csv")

**MERGING FLIGHT AND WEATHER DATA**

In [0]:
flight_df  = da.read_csv("drive/My Drive/flight_final.csv", dtype='unicode')
weather_df = da.read_csv("drive/My Drive/weather_final.csv", dtype='unicode')

In [0]:
flight_df  = flight_df.compute()
weather_df = weather_df.compute()

In [0]:
flight_df['newDep'] = ['0' if i[0:2]=='24' else str(datetime.datetime.strptime(i[0:4],'%H%M%S').time())[0:2]+'00' if i[0]!= '0' else '0' if i[0:2]=='00' else str(datetime.datetime.strptime(i[0:4],'%H%M%S').time())[1:2]+'00' for i in flight_df['DepTime']]

flight_df['newArr'] = ['0' if i[0:2]=='24' else str(datetime.datetime.strptime(i[0:4],'%H%M%S').time())[0:2]+'00' if i[0]!= '0' else '0' if i[0:2]=='00' else str(datetime.datetime.strptime(i[0:4],'%H%M%S').time())[1:2]+'00' for i in flight_df['ArrTime']]

In [0]:
temp_merge = pd.merge(flight_df,weather_df,left_on=['FlightDate','Origin','newDep'],right_on=['date','airport','time'])

temp_merge2 = pd.merge(temp_merge,weather_df ,left_on=['FlightDate','Dest','newArr'],right_on=['date','airport','time'])

In [0]:
temp_merge2 = temp_merge2.drop(columns=['Unnamed: 0_x' ,'Unnamed: 0_y' , 'Unnamed: 0' , 'time_x' ,'time_y' ,'date_x' ,'date_y' ,'airport_x' , 'airport_y' ,'newDep' , 'newArr'])

In [0]:
temp_merge2.to_csv("drive/My Drive/temp_merged.csv")

In [0]:
temp_merge2

,FlightDate,Quarter,Year,Month,DayofMonth,Origin,Dest,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,windspeedKmph_x,DewPointF_x,cloudcover_x,precipMM_x,pressure_x,WindGustKmph_x,visibility_x,weatherCode_x,tempF_x,WindChillF_x,winddirDegree_x,humidity_x,windspeedKmph_y,DewPointF_y,cloudcover_y,precipMM_y,pressure_y,WindGustKmph_y,visibility_y,weatherCode_y,tempF_y,WindChillF_y,winddirDegree_y,humidity_y
0,2016-01-01,1,2016,1,1,SEA,JFK,0741,0.00,0745,0.00,14747,12478,1610,1602,0.00,8.00,7,23,0,0.0,1030,8,10,113,34,29,107,66,22,32,0,0.0,1015,26,10,113,41,32,255,74
1,2016-01-01,1,2016,1,1,MCO,JFK,1408,1.00,1330,38.00,13204,12478,1628,1603,1.00,25.00,9,71,68,0.1,1019,11,7,263,81,79,229,79,22,32,0,0.0,1015,26,10,113,41,32,255,74
2,2016-01-01,1,2016,1,1,MCO,JFK,1445,0.00,1445,0.00,13204,12478,1659,1720,0.00,0.00,9,71,68,0.1,1019,11,7,263,81,79,229,79,22,32,0,0.0,1015,26,10,113,41,32,255,74
3,2016-01-01,1,2016,1,1,LAX,JFK,0853,0.00,0855,0.00,12892,12478,1650,1712,0.00,0.00,12,30,1,0.0,1020,15,10,113,51,47,30,46,22,32,0,0.0,1015,26,10,113,41,32,255,74
4,2016-01-01,1,2016,1,1,LAX,JFK,0842,0.00,0840,2.00,12892,12478,1626,1708,0.00,0.00,12,30,1,0.0,1020,15,10,113,51,47,30,46,22,32,0,0.0,1015,26,10,113,41,32,255,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851428,2017-09-12,3,2017,9,12,JFK,LAX,2237,0.00,2245,0.00,12478,12892,0119,0156,0.00,0.00,14,65,42,0.0,1013,19,10,113,70,70,229,85,7,66,47,0.0,1013,8,10,116,68,68,242,93
1851429,2017-09-16,3,2017,9,16,EWR,MCO,2202,0.00,2155,7.00,11618,13204,0030,0028,0.00,2.00,6,69,78,0.0,1021,7,10,143,70,70,158,95,9,76,13,0.0,1016,13,10,116,81,81,76,85
1851430,2017-09-19,3,2017,9,19,JFK,LAX,0110,1.00,2143,207.00,12478,12892,0354,0059,1.00,175.00,21,68,98,0.0,1017,28,10,122,69,69,51,94,8,62,97,0.0,1012,9,10,122,65,65,145,90
1851431,2017-09-21,3,2017,9,21,JFK,LAX,2358,1.00,2245,73.00,12478,12892,0247,0156,1.00,51.00,19,63,93,0.0,1015,28,10,119,71,71,22,76,11,63,56,0.0,1010,12,10,143,65,65,179,91
